In [1]:
import numpy as np 
import matplotlib

In [2]:
np.random.seed(300)
inputs = np.random.randn(4, 3)
inputs

array([[-1.48517036, -0.26020475, -1.42456537],
       [-0.89139893,  0.76116335, -0.69875726],
       [-0.39664965,  1.24378736,  0.07451846],
       [-0.32294803,  1.46147163,  1.24182139]])

In [3]:
class LayerDense:
    def __init__(self, n_inputs, n_neurons) -> None:
        self.weights = np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros(1, n_neurons)
    
    def forward(self, inputs):
        self.outputs = np.dot(inputs, self.weights) + self.biases

class ActReLU:
    def __init__(self, inputs) -> None:
        self.output = np.maximum(0, inputs)

class SoftMax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        p = exp_values/np.sum(inputs, axis=1, keepdims=True)
        self.output = p

class Loss:
    def calculate_data_loss(self, output, y):
        sample_losses = self.forward
        data_loss = np.mean(sample_losses)
        return data_loss
    
class Loss_categoticalCrossentropy(Loss):
    def forward(self, predictions, y_true):
        n_inputs = len(predictions)
        pclipped = np.clip(predictions, 1e-7, 1-1e-7)

        if len(y_true.shape) == 1:
            correct_p = pclipped[range(n_inputs, y_true)]
        elif len(y_true.shape()) == 2:
            correct_p = np.sum(pclipped*y_true, axis=1)

        nlogp = -np.log(correct_p)
        return nlogp

